# PRIVATE INVESTMENT IN EDUCATION
# Model Ben-Porath (1967)

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [4]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
from modelproject import HumanCapitalModelClass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

**Write out the model in equations here.** 

The model presented by Yed Ben-Porath in 1967 (1) applies theory from capital investment models to study the decision to invest in education in the life-cycle (here two periods). This model treats education is an investment because the individuals need time/resources to invest in it, there are uncertain future returns, and there is depreciation from losing abilities or abilities that start to be obsolete.

The model presents a representative individual whose chooses how much time to invest in education (form 0 to 1) in time t and time t+1, and how much invest in labor market, with the aim of maximizing his utility (life-time utility). Education increases human capital which increases the wages won at the labor market, and the individual is impatience ($\rho$).

The life time utility is:

$$
V =  U_t - C_t + (\frac{1}{1+\rho})[U_{t+1} - C_{t+1}]
$$

Where $U_t$ and $U_{t+1}$ are the utility from study, which is equal to the benefits of studing, i.e. the wage received at labor market which is the return of human capital; this return depends on how much labor market values human capital ($\beta_t$ and $\beta_{t+1}$). Hence, $U_t = \beta_t * H_t$ and $U_{t+1} = \beta_{t+1} * H_{t+1}$.

$C_t$ and $C_{t+1}$ are the cost of studying. These costs come from the direct cost of school such as fees ($\gamma$), which is the same for all individuals, plus the opportunity cost of studying because of not working for a wage. Hence, $C_t= (\gamma * S_t) + (S_t * \beta_t * H_t)$, and $C_{t+1}= (\gamma * S_{t+1}) + (S_{t+1} * \beta_{t+1} * H_{t+1})$

Finally, the human capital acummulation formula is:

$$
H_{t+1}=H_t (1-\delta) + (A*E_t*S_t*H_t)^\alpha
$$

This equation represents a positive relationship between $H_{t+1}$ and $H_t$, which is concave ($\alpha < 1$), meaning a decreasing human capital production. In addition, we have $A$ which is exogeneous initial level of ability, $E_t$ which is the level of government investment in education, $S_t$ is the level of education, and $H_t$ is the previous human capital level.

Including all these equations (utility, cost and human capital accumuation), we have the following final life-utility function.

$$
V = (1-S_t) * \beta_t * H_t  - \gamma * S_t  + (\frac{1}{1+\rho})[(1-S_{t+1})\beta_{t+1} * (H_t (1-\delta) + (A*E_t*S_t*H_t)^\alpha) - \gamma * S_{t+1}]
$$

From this equation, the indidivual chooses $S_t$ and $S_{t+1}$, which have to be between 0 and 1.

## Analytical solution

The model has an analytical solucion:

$$
S_t^* = [\frac{\beta_{t+1}}{\beta_t} * \frac{\alpha}{1+\rho} * \frac{1}{H_t+\gamma/\beta_t} * (A H_t E_t)^\alpha * (1-S_{t+1})]^\frac{1}{1-\alpha}
$$

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

In [14]:
model = HumanCapitalModelClass()

#Numerical solution of the model
solution = model.solve_discrete()

print(solution)



namespace(S0=0.020833333333333332, S1=0.0)


# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 